In [40]:
from liptrf.models.linear_toy import Net
from liptrf.models.vit import ViT

import torch.nn as nn 
import torch 

In [55]:
l2_model = ViT(image_size=28, patch_size=7, num_classes=10, channels=1,
            dim=128, depth=1, heads=8, mlp_ratio=4, attention_type='L2', lmbda=1).cuda()
dp_model = ViT(image_size=28, patch_size=7, num_classes=10, channels=1,
            dim=128, depth=1, heads=8, mlp_ratio=4, attention_type='DP', lmbda=1).cuda()

In [56]:
l2_weight = torch.load('../weights/vit_mnist_layers-1_att-L2.pt')
dp_weight = torch.load('../weights/vit_mnist_layers-1_att-DP.pt')

In [57]:
dp_model.load_state_dict(dp_weight)
dp_model.eval()

ViT(
  (rearrange_patch): Rearrange('b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1=7, p2=7)
  (to_patch_embedding): LinearX()
  (dropout): Dropout(p=0.0, inplace=False)
  (transformer): Transformer(
    (layers): ModuleList(
      (0): ModuleList(
        (0): PreNorm(
          (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
          (fn): Attention(
            (attend): Softmax(dim=-1)
            (to_qkv): LinearX()
            (to_out): LinearX()
            (dropout): Dropout(p=0.0, inplace=False)
          )
        )
        (1): PreNorm(
          (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
          (fn): FeedForward(
            (fc1): LinearX()
            (gelu): GELU()
            (dropout): Dropout(p=0.0, inplace=False)
            (fc2): LinearX()
          )
        )
      )
    )
  )
  (to_latent): Identity()
  (mlp_ln): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  (mlp_head): LinearX()
)

In [58]:
attention_io = {}
def get_activation(name):
    def hook(model, input, output):
        attention_io[name] = {"in": input[0].detach(),
                              "out": output.detach()}
    return hook 

In [59]:
dp_model.transformer.layers[0][0].fn.register_forward_hook(get_activation('a1'))

In [60]:
x = torch.randn(1, 1, 28, 28).cuda()
y = dp_model(x)

In [62]:
attention_io['a1']['in'].shape

torch.Size([1, 17, 128])